In [1]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import warnings
import pandas as pd

# Ignore all warnings
warnings.filterwarnings("ignore")
# Specify the path to your existing CSV file
csv_file_path = 'amazon_validation13.csv'

# Read ASINs from 'ASINS.csv'
asins_file_path = 'ASINs_validation - AE.csv'
with open(asins_file_path, 'r') as asins_file:
    # Assuming ASINs are in the first column of the CSV file
    #asins_list = [row[0] for row in csv.reader(asins_file)]
    # Assuming ASINs are in the first column of the CSV file
    asins_list = [row[0] for row in csv.reader(asins_file) if row]

# Create a list to store failed ASINs
failed_asins_list = []

# Open the CSV file in append mode
with open(csv_file_path, 'w', newline='', encoding='UTF-8') as f:
    writer = csv.writer(f)

    # Check if the file is empty and write headers if needed
    if f.tell() == 0:
        # writer.writerow(['ASIN', 'Title','products_overview','bullet_points','product_information','product_description'])
        writer.writerow(['ASIN', 'Title','buybox_owner','Price','products_overview','bullet_point','product_information','product_description','Review_check','image_block','Bread_crumb'])

    # Iterate through each ASIN and fetch data simultaneously
    for asin in asins_list:
        retry_count = 2  # Number of retries
        while retry_count > 0:
            # Create the URL for the current ASIN
            base_url = "https://www.amazon.ae/dp/"
            url = base_url + asin

            # Make a request to the Amazon product page
            headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Ch": "ect,rtt,downlink,device-memory,sec-ch-device-memory,viewport-width,sec-ch-viewport-width,dpr,sec-ch-dpr,sec-ch-ua-platform-version",
    "Accept-Ch-Lifetime": "86400",
    "Alt-Svc": "h3=\":443\"; ma=86400",
    "Accept-Encoding": "gzip, deflate,br",
    "authority": "www.amazon.ae",
    "method": "GET",
    "path": "/vap/ew/componentbuilder",
    "scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1",
    "Cookie": """session-id=258-2900743-3063138; i18n-prefs=AED; ubid-acbae=262-7166362-0425420; session-token=oEwDvD7tHyNb/9DbAcn21EkWg9fo27ZGj2rc9fLvKb/jh8hselr5THZuKJaQurtdDpnf68WIoUMxkPGDyB4fRBdabPHbyEIyMyioi2qowwUJwUp4Z14NxrxGSbyUG1q84ovLtJM3F0GqQwsFSzvHXVEqR6ORI5ylo+I9gfnnP6Q6Offuzmkx87NX4RmvlT6uf0APzE/WA4qD0A6DzSiur1754ZZqTqsnvarJZ7DUoLWOUGm5KAWE9qIfDxtX/h2AceS+3UspNBamI+MZAjT/3Ht8gSIP3Dc4r28ZuIJqvrTPFCoWraZeJbuPoT+9yyNyhEy8uFhAFZd0o8j4bZpd4VONtEbrNMT8; lc-acbae=en_AE; csm-hit=tb:s-R28AK4NRYTQ3DBBPA64K|1739426930423&t:1739426932208&adb:adblk_no; session-id-time=2082758401l"""
}
            try:
                page = requests.get(url, headers=headers)
                time.sleep(0.8)  # Introduce a delay between requests
            except Exception as e:
                print(f"Error fetching data for ASIN {asin}: {e}")
                # Append the ASIN to the failed_asins_list
                failed_asins_list.append({'ASIN': asin, 'Reason': str(e)})
                continue  # Retry if there is an error

            # Parse the HTML content
            soup1 = BeautifulSoup(page.content, 'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

            # Extract data from the page
            title = soup2.find(id='productTitle').get_text().strip() if soup2.find(id='productTitle') else 'N/A'
            Bread_crumb = soup2.find(id='wayfinding-breadcrumbs_feature_div').get_text().strip() if soup2.find(
                id='wayfinding-breadcrumbs_feature_div') else 'N/A'
            buybox_owner = soup2.find(id= "offer-display-features") if soup2.find(id= "offer-display-features") else 'N/A'
            price = 'N/A'
            unavailable_count = soup2.get_text().count("Currently unavailable")            

            # Check if the item is unavailable
            if unavailable_count >= 2:
                price = 'N/A'
                print(f"ASIN {asin} is unavailable. Setting price to 'N/A'.")
            else:
                # Attempt to find price in various sections
                if price == 'N/A':
                    print("Attempting to find price in multiple sections.")
                    
                    # List of potential sections with price spans
                    potential_ids = [
                        'corePriceDisplay_desktop_feature_div',  # Main target
                        'priceblock_ourprice',                  # Fallback for list price
                        'priceblock_dealprice',                 # For deals
                        'corePrice_feature_div',
                        'aok-offscreen'
                                                    # Another possible section
                    ]
                    
                    price_section = None
                    for section_id in potential_ids:
                        price_section = soup2.find('div', id=section_id)
                        if price_section:
                            break
                    
                    if not price_section:
                        # Check for price in table structure with 'Bundle Price' and 'Bundle List Price'
                        table_section = soup2.find('table', class_='a-lineitem a-align-top')
                        if table_section:
                            # Extract 'Bundle List Price'
                            bundle_list_price_span = table_section.find('span', class_='a-price a-text-price a-size-base')
                            if bundle_list_price_span:
                                bundle_list_price = bundle_list_price_span.find('span', class_='a-offscreen')
                                if bundle_list_price:
                                    price = bundle_list_price.get_text().strip()
                                    print(f"Bundle List Price found: {price}")
                            
                            # Extract 'Bundle Price'
                            if price == 'N/A':  # Only search if Bundle List Price is not found
                                bundle_price_span = table_section.find('span', class_='a-price a-text-price a-size-medium apexPriceToPay')
                                if bundle_price_span:
                                    bundle_price = bundle_price_span.find('span', class_='a-offscreen')
                                    if bundle_price:
                                        price = bundle_price.get_text().strip()
                                        print(f"Bundle Price found: {price}")
                    
                    # Broader search for any price element (fallback)
                    if price == 'N/A':
                        print("Attempting to find price in any span with 'a-price' class.")
                        price_spans = soup2.find_all('span', class_='a-price')
                        for price_span in price_spans:
                            offscreen_price = price_span.find('span', class_='a-offscreen')
                            if offscreen_price and offscreen_price.get_text().strip():
                                price = offscreen_price.get_text().strip()
                                print(f"Fallback price found: {price}")
                                break  # Exit loop once a valid price is found            

                    if price_section:
                        # Search for the price in the section
                        price_block = price_section.find('span', class_='a-price aok-align-center reinventPricePriceToPayMargin priceToPay')
                        if price_block:
                            price_symbol = price_block.find('span', class_='a-price-symbol')
                            price_whole = price_block.find('span', class_='a-price-whole')
                            price_fraction = price_block.find('span', class_='a-price-fraction')            

                            if price_symbol and price_whole and price_fraction:
                                price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
                                print(f"Price found: {price}")
                            else:
                                print("Price components not found in the specified structure.")
                        else:
                            print("Price block not found in the identified section.")
                    else:
                        print("No relevant price section found.")
            products_overview = soup2.find('div', {'data-csa-c-content-id':'productOverview'}) 
            #product_overview = soup2.find(id='productOverview_feature_div').get_text().strip() if soup2.find(id='productOverview_feature_div') else 'N/A'
            bullet_point_element = soup2.find(id='productFactsDesktopExpander') or soup2.find(id='feature-bullets')
            bullet_point = bullet_point_element.get_text().strip() if bullet_point_element else 'N/A'
            product_information = soup2.find(id='prodDetails').get_text().strip() if soup2.find(id='prodDetails') else 'N/A'
            product_description = soup2.find(id='productDescription').get_text().strip() if soup2.find(id='productDescription') else 'N/A'
            Review_check = soup2.find(id='reviewsMedley').get_text().strip() if soup2.find(
                id='reviewsMedley') else 'N/A'
                        # Locate the #imageBlock_feature_div section
            image_block_feature_div = soup2.find("div", id="imageBlock_feature_div")            

             # Locate the desired script within the #imageBlock_feature_div
            image_block_script = (
                    image_block_feature_div.find("script", text=lambda t: t and "P.when('A').register(\"ImageBlockATF\"" in t)
                    if image_block_feature_div
                    else None
            )
            image_block = image_block_script.string.strip() if image_block_script else 'N/A'
            # image_block = image_block_script.text.strip() if image_block_script else 'N/A'
            # imgs_str = img_div.img.get('data-a-dynamic-image')  # a string in Json format
            # Reviews_overall = soup2.find(id='customerReviews').get_text().strip() if soup2.find(
            #    id='customerReviews') else 'N/A'
            # Insights_reviews = soup2.find(id='cr-product-insights-cards').get_text().strip() if soup2.find(
            #    id='cr-product-insights-cards') else 'N/A'
            # Twister_data = soup2.find(id='twister').get_text().strip() if soup2.find(
            #    id='twister') else 'N/A'
                

            # Check if title or ratings is 'N/A' and retry if needed
            if title == 'N/A':
                retry_count -= 1
                print(f"Retrying for ASIN {asin}. Retries left: {retry_count}")
            else:
                break  # Break out of the retry loop if successful

        # If all retries failed, print an error message
        if retry_count == 0:
            print(f"Failed to fetch data for ASIN {asin} after multiple retries.")
            # Append the ASIN to the failed_asins_list
            failed_asins_list.append({'ASIN': asin, 'Reason': 'Max retries reached'})
            continue

        # Write the data to the CSV file
        writer.writerow([asin, title,buybox_owner,price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb])
        # writer.writerow([asin, title, price])

# Convert the failed_asins_list to a DataFrame
failed_asins_df = pd.DataFrame(failed_asins_list)

# Save the failed ASINs to a separate CSV file
failed_asins_df.to_csv('failed_asins-final.csv', index=False)

Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN ﻿B06XYDJM2N. Retries left: 1
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN ﻿B06XYDJM2N. Retries left: 0
Failed to fetch data for ASIN ﻿B06XYDJM2N after multiple retries.
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: AED117.09
Price found: AED117
                 
                  ..09
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: AED107.16
Price found: AED107
                 
                  ..16
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: AED198.52
Price found: AED198
      

In [3]:
failed_asins_df= pd.read_csv('failed_asins-final.csv')
failed_asins_df

,ASIN,Reason
0,﻿B06XYDJM2N,Max retries reached
1,B01B5KYQB2,Max retries reached
2,B00DOPNZMI,Max retries reached
3,B07DDYBMS2,Max retries reached
4,B01ADBR3QE,Max retries reached
...,...,...
1205,B01MYXVU32,Max retries reached
1206,B07GWM9HJZ,Max retries reached
1207,B00NJSYDBC,Max retries reached
1208,B07GWFG9D9,Max retries reached


In [2]:
import pandas as pd
validation_df1=pd.read_csv('amazon_validation13.csv')

pd.set_option("display.max_columns", None)
validation_df1


,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,B06XYK6Y76,"Restaurantware Mixing Bowl, Metal German Bowl ...","<div class=""a-section a-spacing-base offer-dis...",AED117\n \n ..09,"<div class=""celwidget"" data-csa-c-asin=""B06XYK...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,"Prepare recipes, ingredients, sauces, salads a...",Customer Reviews\n \n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
1,B012ATFRBA,Restaurantware RWA0019 Conetek 10-Inch Eco-Fri...,"<div class=""a-section a-spacing-base offer-dis...",AED107\n \n ..16,"<div class=""celwidget"" data-csa-c-asin=""B012AT...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,"Our Black Cone Tek Food Cones add a simple, co...",Customer Reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
2,B00XJDSKMU,"Restaurantware Duet Tasting Cup, Double Sided ...","<div class=""a-section a-spacing-base offer-dis...",AED198\n \n ..52,"<div class=""celwidget"" data-csa-c-asin=""B00XJD...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,A charming and fun way to serve two flavors of...,Customer Reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
3,B013K0PBSC,Restaurantware Single Use Place Mat - Heavy We...,"<div class=""a-section a-spacing-base offer-dis...",AED646\n \n ..24,"<div class=""celwidget"" data-csa-c-asin=""B013K0...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Give your place settings a sophisticated touch...,Customer Reviews\n \n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Home\n \n\n\n\n\n ...
4,B08W1HHTPJ,"3.5 Inch Snowflake Cocktail Picks, 1000 Sturdy...",NaN,NaN,"<div class=""celwidget"" data-csa-c-asin=""B08W1H...",About this item\n \n\n\n\n ...,Product information\n \n\n\n\n\n\n\n...,Give your snacks the perfect holiday touch wit...,Customer Reviews\n \n\n\n\n...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
...,...,...,...,...,...,...,...,...,...,...,...
1787,B075XZ22CM,Restaurantware 16-inch Stainless Steel Barspoo...,"<div class=""a-section a-spacing-base offer-dis...",AED62\n \n ..50,"<div class=""celwidget"" data-csa-c-asin=""B075XZ...",About this item\n \n\n\n\n ...,NaN,NaN,Customer Reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
1788,B0183UXTYM,Restaurantware Porcelain Bowl Set with Tray - ...,"<div class=""a-section a-spacing-base offer-dis...",AED157\n \n ..68,"<div class=""celwidget"" data-csa-c-asin=""B0183U...",About this item\n \n\n\n\n ...,NaN,NaN,Customer Reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
1789,B07GWGLQ6H,"Restaurantware Cake Decorating Icing Tip, Cupc...","<div class=""a-section a-spacing-base offer-dis...",AED22\n \n ..28,"<div class=""celwidget"" data-csa-c-asin=""B07GWG...",About this item\n \n\n\n\n ...,NaN,NaN,Customer Reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
1790,B07GWG3FVP,Restaurantware Gray Cotton Oven Mitt - Flame R...,"<div class=""a-section a-spacing-base offer-dis...",AED116\n \n ..35,"<div class=""celwidget"" data-csa-c-asin=""B07GWG...",About this item\n \n\n\n\n ...,NaN,NaN,Customer Reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Tools & Home Improvement\n \n\n\...


# Retrying Failed ASINs

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import warnings
import pandas as pd

# Ignore all warnings
warnings.filterwarnings("ignore")
# Specify the path to your existing CSV file
csv_file_path = 'amazon_validation14.csv'

# Read ASINs from 'ASINS.csv'
asins_file_path = 'failed_asins-final.csv'
with open(asins_file_path, 'r') as asins_file:
    # Assuming ASINs are in the first column of the CSV file
    #asins_list = [row[0] for row in csv.reader(asins_file)]
    # Assuming ASINs are in the first column of the CSV file
    asins_list = [row[0] for row in csv.reader(asins_file) if row]

# Create a list to store failed ASINs
failed_asins_list = []

# Open the CSV file in append mode
with open(csv_file_path, 'w', newline='', encoding='UTF-8') as f:
    writer = csv.writer(f)

    # Check if the file is empty and write headers if needed
    if f.tell() == 0:
        # writer.writerow(['ASIN', 'Title','products_overview','bullet_points','product_information','product_description'])
        writer.writerow(['ASIN', 'Title','buybox_owner','Price','products_overview','bullet_point','product_information','product_description','Review_check','image_block','Bread_crumb'])

    # Iterate through each ASIN and fetch data simultaneously
    for asin in asins_list:
        retry_count = 2  # Number of retries
        while retry_count > 0:
            # Create the URL for the current ASIN
            base_url = "https://www.amazon.ae/dp/"
            url = base_url + asin

            # Make a request to the Amazon product page
            headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Ch": "ect,rtt,downlink,device-memory,sec-ch-device-memory,viewport-width,sec-ch-viewport-width,dpr,sec-ch-dpr,sec-ch-ua-platform-version",
    "Accept-Ch-Lifetime": "86400",
    "Alt-Svc": "h3=\":443\"; ma=86400",
    "Accept-Encoding": "gzip, deflate,br",
    "authority": "www.amazon.ae",
    "method": "POST",
    "path": "/vap/ew/componentbuilder",
    "scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1",
    "Cookie": """session-id=258-2900743-3063138; i18n-prefs=AED; ubid-acbae=262-7166362-0425420; lc-acbae=en_AE; session-token=krEV9TCJzbodu5t9aIbpDjFrSMQbM478pu9sYRK9go88iHjSyWdqOr1mAzF5JJryMX4AAX+/ZrKMTE45s4IOPhT7FI1RbHn6VACLcOU1lIFF34ZtevqLWwmDLGZSzIsfVWgcWmX2GgoQK5TPWIPjG2HEY9QApGZun5wz3ZH0aE+H+qj+Dmu0iiDQN7hfEwyNBrusEwBzOzHfE8FZ8j9J6TU2OAwo9KeTlt6cTub58SNKHjHDE++gFMRfTPf/qyKb788ah6yvShThIntpukTAVidlQ67Qri9o2DLBrJdc4VnMuY1n6r+sCHxXk1VQXlBcYyJdoayI+5sl/LEEDgkDUeyvCwwhn+Fr; session-id-time=2082787201l; csm-hit=tb:s-DR5N1WV9EW6JB8GY3AJ6|1739438902975&t:1739438905947&adb:adblk_no"""
}
            try:
                page = requests.get(url, headers=headers)
                time.sleep(1.8)  # Introduce a delay between requests
            except Exception as e:
                print(f"Error fetching data for ASIN {asin}: {e}")
                # Append the ASIN to the failed_asins_list
                failed_asins_list.append({'ASIN': asin, 'Reason': str(e)})
                continue  # Retry if there is an error

            # Parse the HTML content
            soup1 = BeautifulSoup(page.content, 'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

            # Extract data from the page
            title = soup2.find(id='productTitle').get_text().strip() if soup2.find(id='productTitle') else 'N/A'
            Bread_crumb = soup2.find(id='wayfinding-breadcrumbs_feature_div').get_text().strip() if soup2.find(
                id='wayfinding-breadcrumbs_feature_div') else 'N/A'
            buybox_owner = soup2.find(id= "offer-display-features") if soup2.find(id= "offer-display-features") else 'N/A'
            price = 'N/A'
            unavailable_count = soup2.get_text().count("Currently unavailable")            

            # Check if the item is unavailable
            if unavailable_count >= 2:
                price = 'N/A'
                print(f"ASIN {asin} is unavailable. Setting price to 'N/A'.")
            else:
                # Attempt to find price in various sections
                if price == 'N/A':
                    print("Attempting to find price in multiple sections.")
                    
                    # List of potential sections with price spans
                    potential_ids = [
                        'corePriceDisplay_desktop_feature_div',  # Main target
                        'priceblock_ourprice',                  # Fallback for list price
                        'priceblock_dealprice',                 # For deals
                        'corePrice_feature_div',
                        'aok-offscreen'
                                                    # Another possible section
                    ]
                    
                    price_section = None
                    for section_id in potential_ids:
                        price_section = soup2.find('div', id=section_id)
                        if price_section:
                            break
                    
                    if not price_section:
                        # Check for price in table structure with 'Bundle Price' and 'Bundle List Price'
                        table_section = soup2.find('table', class_='a-lineitem a-align-top')
                        if table_section:
                            # Extract 'Bundle List Price'
                            bundle_list_price_span = table_section.find('span', class_='a-price a-text-price a-size-base')
                            if bundle_list_price_span:
                                bundle_list_price = bundle_list_price_span.find('span', class_='a-offscreen')
                                if bundle_list_price:
                                    price = bundle_list_price.get_text().strip()
                                    print(f"Bundle List Price found: {price}")
                            
                            # Extract 'Bundle Price'
                            if price == 'N/A':  # Only search if Bundle List Price is not found
                                bundle_price_span = table_section.find('span', class_='a-price a-text-price a-size-medium apexPriceToPay')
                                if bundle_price_span:
                                    bundle_price = bundle_price_span.find('span', class_='a-offscreen')
                                    if bundle_price:
                                        price = bundle_price.get_text().strip()
                                        print(f"Bundle Price found: {price}")
                    
                    # Broader search for any price element (fallback)
                    if price == 'N/A':
                        print("Attempting to find price in any span with 'a-price' class.")
                        price_spans = soup2.find_all('span', class_='a-price')
                        for price_span in price_spans:
                            offscreen_price = price_span.find('span', class_='a-offscreen')
                            if offscreen_price and offscreen_price.get_text().strip():
                                price = offscreen_price.get_text().strip()
                                print(f"Fallback price found: {price}")
                                break  # Exit loop once a valid price is found            

                    if price_section:
                        # Search for the price in the section
                        price_block = price_section.find('span', class_='a-price aok-align-center reinventPricePriceToPayMargin priceToPay')
                        if price_block:
                            price_symbol = price_block.find('span', class_='a-price-symbol')
                            price_whole = price_block.find('span', class_='a-price-whole')
                            price_fraction = price_block.find('span', class_='a-price-fraction')            

                            if price_symbol and price_whole and price_fraction:
                                price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
                                print(f"Price found: {price}")
                            else:
                                print("Price components not found in the specified structure.")
                        else:
                            print("Price block not found in the identified section.")
                    else:
                        print("No relevant price section found.")
            products_overview = soup2.find('div', {'data-csa-c-content-id':'productOverview'}) 
            #product_overview = soup2.find(id='productOverview_feature_div').get_text().strip() if soup2.find(id='productOverview_feature_div') else 'N/A'
            bullet_point_element = soup2.find(id='productFactsDesktopExpander') or soup2.find(id='feature-bullets')
            bullet_point = bullet_point_element.get_text().strip() if bullet_point_element else 'N/A'
            product_information = soup2.find(id='prodDetails').get_text().strip() if soup2.find(id='prodDetails') else 'N/A'
            product_description = soup2.find(id='productDescription').get_text().strip() if soup2.find(id='productDescription') else 'N/A'
            Review_check = soup2.find(id='reviewsMedley').get_text().strip() if soup2.find(
                id='reviewsMedley') else 'N/A'
                        # Locate the #imageBlock_feature_div section
            image_block_feature_div = soup2.find("div", id="imageBlock_feature_div")            

             # Locate the desired script within the #imageBlock_feature_div
            image_block_script = (
                    image_block_feature_div.find("script", text=lambda t: t and "P.when('A').register(\"ImageBlockATF\"" in t)
                    if image_block_feature_div
                    else None
            )
            image_block = image_block_script.string.strip() if image_block_script else 'N/A'
            # image_block = image_block_script.text.strip() if image_block_script else 'N/A'
            # imgs_str = img_div.img.get('data-a-dynamic-image')  # a string in Json format
            # Reviews_overall = soup2.find(id='customerReviews').get_text().strip() if soup2.find(
            #    id='customerReviews') else 'N/A'
            # Insights_reviews = soup2.find(id='cr-product-insights-cards').get_text().strip() if soup2.find(
            #    id='cr-product-insights-cards') else 'N/A'
            # Twister_data = soup2.find(id='twister').get_text().strip() if soup2.find(
            #    id='twister') else 'N/A'
                

            # Check if title or ratings is 'N/A' and retry if needed
            if title == 'N/A':
                retry_count -= 4
                print(f"Retrying for ASIN {asin}. Retries left: {retry_count}")
            else:
                break  # Break out of the retry loop if successful

        # If all retries failed, print an error message
        if retry_count == 0:
            print(f"Failed to fetch data for ASIN {asin} after multiple retries.")
            # Append the ASIN to the failed_asins_list
            failed_asins_list.append({'ASIN': asin, 'Reason': 'Max retries reached'})
            continue

        # Write the data to the CSV file
        writer.writerow([asin, title,buybox_owner,price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb])
        # writer.writerow([asin, title, price])

# Convert the failed_asins_list to a DataFrame
failed_asins_df = pd.DataFrame(failed_asins_list)

# Save the failed ASINs to a separate CSV file
failed_asins_df.to_csv('failed_asins-final-2.csv', index=False)

Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN ASIN. Retries left: -4
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN ﻿B06XYDJM2N. Retries left: -4
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN B01B5KYQB2. Retries left: -4
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
No relevant price section found.
Retrying for ASIN B00DOPNZMI. Retries left: -4
Attempting to find price in multiple sections.
Attempting to find price in any span with 'a-price' class.
Fallback price found: AED62.12
Price found: AED62
                 
                  ..12
Attempting to find price in multiple sections.
Attempting to fin

KeyboardInterrupt: 

In [7]:
import pandas as pd
validation_df1=pd.read_csv('amazon_validation14.csv')

pd.set_option("display.max_columns", None)
validation_df1


,ASIN,Title,buybox_owner,Price,products_overview,bullet_point,product_information,product_description,Review_check,image_block,Bread_crumb
0,ASIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,﻿B06XYDJM2N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B01B5KYQB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B00DOPNZMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B07DDYBMS2,"Restaurantware Washable Paper Bags, Reusable B...","<div class=""a-section a-spacing-base offer-dis...",AED62\n \n ..12,"<div class=""celwidget"" data-csa-c-asin=""B07DDY...",About this item\n \n\n\n\n ...,NaN,NaN,Customer Reviews\n \n\n\n\n\...,"P.when('A').register(""ImageBlockATF"", function...",Kitchen\n \n\n\n\n\n ...
5,B01ADBR3QE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,B01ADBRO4U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,B075S63C9G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,B0183UXWJY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,B00DOOFZGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
